In [40]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer

### Metrique d'évaluation utilisé
Log Likelihood : Plus la vraisemblance logarithmique est élevée, meilleur est le modèle. Vous pouvez obtenir ce score en utilisant la méthode score de l'objet LDA.  
  
Perplexity : La perplexité est une mesure de la qualité du modèle. Une perplexité plus faible indique un meilleur modèle. Vous pouvez obtenir ce score en utilisant la méthode perplexity.  
  
Coherence Score : Le score de cohérence mesure la cohérence sémantique des sujets générés par le modèle LDA. Un score de cohérence plus élevé indique que les mots les plus importants d'un sujet sont plus sémantiquement similaires les uns aux autres. Le score de cohérence n'est pas directement disponible dans scikit-learn, mais peut être calculé en utilisant la bibliothèque Gensim.





In [41]:

# Pipeline avec les vectoriasations bag of words et tf-idf

class CustomCountVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.vectorizer = CountVectorizer(min_df=0.10, max_df=0.5, ngram_range=(1, 1), stop_words='english')

    def fit(self, X, y=None):
        self.vectorizer.fit(X)
        return self

    def transform(self, X, y=None):
        return self.vectorizer.transform(X)
    
class CustomTfidfVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.vectorizer = TfidfVectorizer(min_df=0.10, max_df=0.2, ngram_range=(1, 1), stop_words='english')

    def fit(self, X, y=None):
        self.vectorizer.fit(X)
        return self

    def transform(self, X, y=None):
        return self.vectorizer.transform(X)

pipeline_vec = Pipeline([
    ("count_vectorizer", CustomCountVectorizer())
])


def create_vectors_pipeline():
    pipeline_vec = Pipeline([
        ("count_vectorizer", CustomCountVectorizer())
    ])
    return pipeline_vec

def create_tfidf_pipeline():
    pipeline_tfidf = Pipeline([
        
        ("tfidf", CustomTfidfVectorizer())
    ])
    return pipeline_tfidf





### Test avec CountVectorizer

Vectorizer la colonne Body

In [42]:
# Créer le pipeline de vectorisation
pipeline_vec = create_vectors_pipeline()

# Supposons que vous avez une DataFrame pandas X_fulltrain contenant les données textuelles dans une colonne 'body'
X_fulltrain = pd.read_csv("Data/X_train.csv", index_col=0, lineterminator='\n')

# Transformer les données textuelles en utilisant le pipeline
transformed_data_body = pipeline_vec.fit_transform(X_fulltrain.body)
print(transformed_data_body.shape)
# Obtenir le transformateur CustomCountVectorizer du pipeline
custom_count_vectorizer_body = pipeline_vec.named_steps['count_vectorizer']

# Obtenir les noms des caractéristiques (mots)
feature_names_body = custom_count_vectorizer_body.vectorizer.get_feature_names_out()

# Afficher les noms des caractéristiques
print(feature_names_body.shape)


(799, 58)
(58,)


Vectorizer la colonne title

In [43]:
pipeline_vec_title = create_vectors_pipeline()
# Supposons que vous avez une DataFrame pandas X_fulltrain contenant les données textuelles dans une colonne 'body'
X_fulltrain = pd.read_csv("Data/X_train.csv", index_col=0, lineterminator='\n')

# Transformer les données textuelles en utilisant le pipeline
transformed_data_title = pipeline_vec_title.fit_transform(X_fulltrain.title)
print(transformed_data_title.shape)
# Obtenir le transformateur CustomCountVectorizer du pipeline
custom_count_vectorizer_title = pipeline_vec_title.named_steps['count_vectorizer']

# Obtenir les noms des caractéristiques (mots)
feature_names = custom_count_vectorizer_title.vectorizer.get_feature_names_out()

# Afficher les noms des caractéristiques
print(feature_names.shape)

feature_names = np.concatenate((feature_names, feature_names_body))

(799, 1)
(1,)


In [44]:
# Concaténer les deux matrices
transformed_data = np.concatenate((transformed_data_title.toarray(), transformed_data_body.toarray()), axis=1)

# Vérifier la forme de la matrice combinée
print(transformed_data.shape)

(799, 59)


Appliquer LDA sur les données transformé

In [45]:
from sklearn.decomposition import LatentDirichletAllocation

# Créer une instance de LDA
lda = LatentDirichletAllocation(n_components=20, 
                                n_jobs=-1,
                                random_state=42)

# Ajuster le modèle LDA aux données
lda.fit(transformed_data)

# Obtenir les sujets pour chaque document
document_topics = lda.transform(transformed_data)
print(document_topics)
# Obtenir les composants du modèle LDA
components = lda.components_
print(components)
# Obtenir le score de vraisemblance logarithmique
log_likelihood = lda.score(transformed_data)

# Obtenir la perplexité
perplexity = lda.perplexity(transformed_data)

print("Log Likelihood: ", log_likelihood)
print("Perplexity: ", perplexity)
# Pour chaque sujet, imprimer les mots les plus importants
for i, topic in enumerate(components):
    print(f"-------Topic {i}:--------")
    # Obtenir les indices des mots les plus importants pour ce sujet
    word_indices = topic.argsort()[-10:]
    # Imprimer les mots correspondants
    for index in word_indices[:10]:
        print(f"{feature_names[index]}")

[[0.04766254 0.002      0.002      ... 0.002      0.002      0.002     ]
 [0.00263158 0.00263158 0.00263158 ... 0.00263158 0.00263158 0.00263158]
 [0.12405767 0.003125   0.003125   ... 0.07584285 0.003125   0.003125  ]
 ...
 [0.00263158 0.00263158 0.00263158 ... 0.00263158 0.07803832 0.00263158]
 [0.00714286 0.48188378 0.00714286 ... 0.00714286 0.00714286 0.38954479]
 [0.21445541 0.00555556 0.00555556 ... 0.15544558 0.00555556 0.00555556]]
[[1.05360788e+01 5.00000009e-02 5.00000011e-02 ... 4.33241634e+00
  5.00000010e-02 3.11210363e+01]
 [5.00000007e-02 5.00000010e-02 5.00000190e-02 ... 7.52475798e-01
  5.00000005e-02 5.44482549e-02]
 [5.00000006e-02 5.00000007e-02 5.00000009e-02 ... 7.23394350e-01
  5.00000003e-02 1.06296633e+01]
 ...
 [9.05724306e+01 4.17836081e+00 6.70520197e+00 ... 2.23884189e+01
  5.00000011e-02 8.41778290e+00]
 [5.00000004e-02 5.00000002e-02 5.00000007e-02 ... 1.22247629e+01
  2.78744835e+01 1.19832728e+01]
 [5.00000009e-02 5.00000008e-02 2.41225334e+00 ... 8.177

In [46]:
from gensim.models import CoherenceModel

# Convertir les données transformées en une liste de listes de mots
data_words = [list(filter(None, [feature_names[index] for index in row.nonzero()[0]])) for row in transformed_data]
# Convertir les données transformées en une liste de listes de mots
'''data_words = []
for row in transformed_data:
    # Obtenir les indices des valeurs non nulles dans la ligne
    nonzero_indices = row.nonzero()[0]
    # Filtrer les indices pour qu'ils soient dans la plage valide
    valid_indices = [index for index in nonzero_indices if index < len(feature_names)]
    # Ajouter les mots correspondant aux indices valides à la liste
    words = [feature_names[index] for index in valid_indices]
    data_words.append(words)'''

# Créer le modèle LDA de Gensim
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

# Créer un dictionnaire Gensim à partir des données
dictionary = Dictionary(data_words)

# Créer un corpus Gensim
corpus = [dictionary.doc2bow(text) for text in data_words]

# Créer le modèle LDA Gensim
lda_gensim = LdaModel(corpus=corpus,
                      id2word=dictionary,
                      num_topics=20,
                      random_state=42,
                      passes=10,
                      iterations=100)

# Calculer le score de cohérence
coherence_model_lda = CoherenceModel(model=lda_gensim, texts=data_words, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print("Coherence Score:", coherence_lda)
print("Log Likelihood: ", log_likelihood)
print("Perplexity: ", perplexity)

Coherence Score: 0.542213594812194
Log Likelihood:  -57529.9825702939
Perplexity:  45.967356523387444


## Test avec la vectorisation Tf_Idf

In [47]:
# Créer le pipeline de vectorisation
pipeline_tfidf = create_tfidf_pipeline()

# Supposons que vous avez une DataFrame pandas X_fulltrain contenant les données textuelles dans une colonne 'body'
X_fulltrain = pd.read_csv("Data/X_train.csv", index_col=0, lineterminator='\n')

# Transformer les données textuelles en utilisant le pipeline
transformed_data_body_tf = pipeline_tfidf.fit_transform(X_fulltrain.body)
print(transformed_data_body_tf.shape)
# Obtenir le transformateur CustomCountVectorizer du pipeline
custom_count_tf_body = pipeline_tfidf.named_steps['tfidf']

# Obtenir les noms des caractéristiques (mots)
feature_names_body = custom_count_tf_body.vectorizer.get_feature_names_out()

# Afficher les noms des caractéristiques
print(feature_names_body.shape)
print(feature_names_body)
print(transformed_data_body_tf)

(799, 46)
(46,)
['activity' 'add' 'android' 'app' 'application' 'case' 'change' 'class'
 'data' 'doesnt' 'dont' 'example' 'file' 'fragment' 'function' 'google'
 'help' 'implement' 'issue' 'ive' 'know' 'look' 'make' 'method' 'need'
 'override' 'project' 'public' 'question' 'return' 'run' 'say' 'set'
 'solution' 'start' 'string' 'support' 'thanks' 'time' 'true' 'update'
 'user' 'version' 'view' 'void' 'way']
  (0, 34)	0.8193829014871954
  (0, 33)	0.21759777687243947
  (0, 22)	0.1900043940757113
  (0, 21)	0.20670959394617275
  (0, 20)	0.19201234923918767
  (0, 19)	0.19355746020090633
  (0, 18)	0.20608271543489093
  (0, 13)	0.2253890821008403
  (0, 3)	0.18662471878631928
  (1, 39)	0.31183328805782573
  (1, 35)	0.915181764500241
  (1, 3)	0.2553474855667677
  (2, 34)	0.2693488825349901
  (2, 29)	0.24539032663369487
  (2, 13)	0.5927221306596637
  (2, 8)	0.5573540238560171
  (2, 3)	0.24539032663369487
  (2, 2)	0.2524744492794455
  (2, 0)	0.285152546972993
  (3, 39)	0.6501783162322662
  (3, 22)